# Grain Growth Cellular Automata Model

This is an advanced 2D Cellular Automata (CA) model for the simulation of microstructural evolution during grain growth in polycrystalline materials. The model is built on physical principles and it can directly work with experimental EBSD data. It is written in Dyalog APL.

## Features

This model incorporates a sophisticated set of features to achieve a balance between physical realism and computational efficiency:

* **Crystallography:** Crystallographic orientations are represented as Euler angles (Bunge convention) to the user, and internally as quaternions. Misorientations are calculated considering cubic crystal symmetry

* **Moore Neighborhood:** Boundary energy densities are calculated from the contribution of 8 neighbours

    $$\gamma_o=\sum_n\gamma_n=\gamma_{NW}+\gamma_N+\gamma_{NE}+\gamma_E+\gamma_{SE}+\gamma_S+\gamma_{SW}+\gamma_W$$

* **Robust Simulation Mechanics:** The model calculates the transformation rate ($r_n$) for a cell to flip to the orientation of neighbour $p$ (the *parent* cell) based on the equation

    $$r_n = \left(\frac{k_r}{\Delta x^2 N}\right) M_n K |\Delta\gamma_n|$$

    The rate is scaled by cell area $\Delta x^2$ and number of neighbours $N$ to ensure physical kinetics are independent of grid resolution, and by an heuristic factor $K$ (expained below). The $k_r$ parameter, configurable by the user, works as an overall scaling factor for the model.
    
    Each time step, an accumulator of transformation into each neighbor is updated such that $a_n{t+\Delta t}=a_n(t) + r_n \Delta t$. The cell is then transformed or not interpreting this accumulator as a probability (scaled by the user configurable factor $\xi_f$, such that a higher $\xi_f$ implies higher randomness). Therefore, the condition to transform into neighbor $n$ is
    
    $$\xi(0,1)<a_n(t)^{\frac{1}{\xi_f}}$$
    
* **Physically-Based Kinetics:**

    * *Grain boundary energy* is calculated using the Read-Shockley equation, dependent on the misorientation angle with the neighbor $n$, given by $\theta_n$
    
    $$\gamma_n=\gamma_0 \frac{\theta_{on}}{\theta_H}\left(1 - \log\left(\frac{\theta_{on}}{\theta_H}\right)\right)$$


    * *Grain boundary mobility* is a function of both temperature and misorientation, according to Humphrey's equation
    
    $$M_n=M_0 \exp\left(\frac{-Q_g}{R T}\right) \left(1-\exp\left(-A_h\left(\frac{\theta_{on}}{\theta_H}\right)^n_h\right)\right)$$
    
* **Flexible Local Factors:** The driving force can be modified by any number of user-provided local property maps. The following factors are already predefined:

    * *Thermal Energy:* Adds a deterministic driving force for recrystallization
    
    $$\Delta\gamma_T=w_T\frac{k T}{\Delta x^2} \xi(-1,1)$$

    
    * *Image Quality:* Higher energy push at cells with more deffects (more image quality, also related with stored energy)
    
    $$\Delta\gamma_{IQ}=w_{IQ}\gamma_0 \left(1 - \frac{IQ_o}{IQ_{max}}\right)$$
    
    * *Confidence Index:* Higher energy fluctuations at cells with a more uncertain orientation (a lower confidence index)
    
    $$\Delta\gamma_{CI}=w_{CI}(1 - CI_o) \xi(-1,1)$$

    The total transformation energy for each neighbor $n$ is then calculated as:
    
    $$\Delta\gamma_n=(\gamma_n - \gamma_0)+\Delta\gamma_T+\Delta\gamma_{IQ}+\Delta\gamma_{CI}$$

* **Advanced Heuristics:** The factor $K$ is obtained as the product of three components

    $$K=K_c K_m K_g$$

    * *Cooperative factor:* For each neighbours for which there is a misorientation with respect to one of its neighbours of the same angle as the misorientation between the parent and transforming cell ($\delta_{np}$ is 1), it is considered that the boundary energy after transition will be decreased by a factor $k_c$
    
    $$K_c=1-\delta_n\left(1 - k_c\right)$$
    
    * *Geometric factor:* This factor accounts for the influence of the boundary topology in the neighborhood, considering both coherent front advancement and the "squeeze" pressure on shrinking grains, adding the contribution of all moving boundaries
    
    $$K_g=\sqrt{\left(\sum_n{|u_{nx}|}\right)^2+\left(\sum_n{|u_{ny}|}\right)^2}$$
    
    * *Multiple-joint factor:* The factor $K_m$ takes different values depending if the cell is part of a simple boundary, triple point or quadriple point (the quadruple point factor is also used when more than four grains meet)
    
* **Boundary Conditions:** Supports both periodic and "mirror" (no-flux) boundary conditions for realistic simulation of finite samples

* **EBSD Integration:** Can directly read `.ctf` (HKL) and `.ang` (TSL) EBSD files, clean non-indexed points, and crop regions of interest

### Symbols

| Symbol | Description |
| :--- | :--- |
| **Grid** | |
| $\Delta x$ | Cell size (square grid) |
| $N$ | Number of interacting neighbors for a cell (8 for Moore) |
| **Process** | |
| $T$ | Temperature of the process |
| $\Delta t$ | Time increment |
| **Orienations and Misorientations**| |
| $q_o, q_n$ | Crystallographic orientation of cell $o$ and neighbour $n$ |
| $\Delta\theta_{on}$ | Misorientation angle between cell $o$ and neighbour $n$ |
| $\Delta\theta_H$ | Misorientation angle at high angle grain boundaries |
| $\delta_{np}$ | A boolean factor (1 or 0) indicating if there exists some neighbor of $n$, $m$, such that $\theta_{op}=\theta_{nm}$ |
| **Boundary Energy and Mobility**| |
| $\gamma_o$ | Initial grain boundary energy density of cell $o$ |
| $\gamma_n$ | Grain boundary energy density of cell $o$ if it transforms into $n$ |
| $\gamma_0$ | Energy density at HAGB |
| $M_n$ | Mobility of boundary with neighbour $n$ |
| $M_0$ | Reference boundary mobility |
| $Q_g$ | Activation energy |
| $A_h, n_h$ | Factor and exponent in Humphrey's law |
| **Transformation**| |
| $r_n$ | Rate of transformation of cell $o$ into neighbour $n$ |
| $a_n$ | Accumulated transformation of cell $o$ into neighbour $n$ |
| $k_r$ | Transformation rate overall scaling factor |
| $\xi_f$ | Randomness in transformation of cells |
| **Local Factors** | |
| $\Delta\gamma_T, \Delta\gamma_{IQ}, \Delta\gamma_{CI} $ | Contribution to driving force of temperature, image quality, and confidence index |
| $w_T, w_{IQ}, w_{CI}$ | Scaling factors |
| $IQ_o, IQ_{max}$ | Image quality of each cell and maximum image quality |
| $CI_o$ | Confidence index of each cell |
| **Heuristics** | |
| $K$ | Total multiplicative factor |
| $K_c, K_m, K_g$ | Multiplicative factors for cooperative movement, multiple joints and geometry |
| $k_c$ | Cooperative factor (contribution of similar boundaries) |
| $u_{nx}, u_{ny}$ | Components of unitary vector normal to boundary with neighbour $n$ |
| **Others** | |
| $k$ | Boltzman constant |
| $R$ | Universal gas constant |
| $\xi(a,b)$ | Random number between $a$ and $b$ |

## Usage

The model is provided as a `.apls` script. To run it, specify the `.json5` configuration file(s) as argument(s). If no file name is given, the default file `gg2.json5` is used. See this file for further details about configuration options.


In [1]:
)clear
⎕IO←0 ⋄ ⎕RL←1

clear ws

In [2]:
:Namespace Euler                                                     ⍝ Orientations and misorientations
    c←↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)                          ⍝ product components ⎕IO=0
    u←↑(1 ¯1 ¯1 ¯1)(1 1 1 ¯1)(1 ¯1 1 1)(1 1 ¯1 1)                    ⍝ product unit factors
    QP←+/u×⊣(×⍤1)(⊂c)⌷⊢ ⋄ QC←×∘(1,-3⍴1) ⋄ QD←+.×                     ⍝ product, conjugate and dot product
    RD←180÷⍨○ ⋄ UV←×∘(÷(÷2)*⍨+.×⍨)⍨ ⋄ (x y z)←=∘⊂⍨⍳3                 ⍝ degrees, unitary vector and unit vectors
    QA←((⊂2○⊢),(⊂1○⊢)∘.×∘UV{⍺←z ⋄ ⍺})∘(÷∘2) ⋄ QAD←QA∘RD              ⍝ quaternion from axis-angle
    QE←1(×⍤↑×⊢)∘⊃{⍺←z x z ⋄ ⍺}QP.QA⌽ ⋄ QED←QE∘RD                     ⍝ quaternion from Euler angles (zxz)
    cs ← ⊂1 0 0 0                                                    ⍝ cubic symmetry: identity
    cs,←,(1 0 0)(0 1 0)(0 0 1)∘.QAD 90 180 270                       ⍝                 4-fold around <001>
    cs,←,(1 1 1)(¯1 1 1)(1 ¯1 1)(1 1 ¯1)∘.QAD 120 240                ⍝                 3-fold around <111>
    cs,←,(1 1 0)(1 0 1)(0 1 1)(1 ¯1 0)(¯1 0 1)(0 1 ¯1)∘.QAD 180      ⍝                 2-fold around <110>
    MC←⊃2×¯2○1⌊cs⌈.(|QD)∘⊂QC⍤QP∘QC ⋄ DL←⌊○⍤⊣÷⍨180×⊢                  ⍝ misorientation (cubic symmetry) and level
    M←{⍺←0.5                                                         ⍝ namespace to calculate misorientations
        IM←⌈(⊢+2÷⍨⊣×1-⍨⊣)⌊ ⋄ CM←{(c←⎕NS⍬).m←¯1⍴⍨1+⍵IM.-1 2 ⋄ c}      ⍝ index and cache of misorientations
        _M_ ← {(c s)←⍺⍺ ⍵⍵                                           ⍝ memoization
            (a w)←(⊂,d←⍺≠⍵)/∘,¨⍺ ⍵ ⋄ 0=≢a:d                          ⍝   return zeros if no different pairs
            n←(≠,i)∧s.m[i←a IM w]<0 ⋄ 0=≢p←n/⍥,i:s.m[i]@⊢d           ⍝   return calculated if all done
            s.m[p]←DL⊃a MC⍥{↓⍉↑c[n/⍵]}w ⋄ s.m[i]@⊢d}                 ⍝   calculate and return
        m←⎕NS'MC' 'IM' ⋄ m.DL←⍺∘DL ⋄ m.L←⌊÷∘⍺                        ⍝ namespace with curried DL
        m.D←(⍺÷2)+⍺∘× ⋄ w←⍵ ⋄ m.M←w _M_(CM≢w) ⋄ m                    ⍝ degrees and misorientation functions
    }
    RND←{F←{(1 2○2×⊂○?⍺⍴0)×⊂⍵*÷2} ⋄ 1(×⍤↑×⊢)UV ⍵(F∘(1∘-),F)?⍵⍴0}     ⍝ random orientations (quaternions)
    A2←12○⊢+0J1×⊣                                                    ⍝ arctan2
    EA←{                                                             ⍝ euler angles from quaternion
        p←¯2○1-⍨2×(+/×⍨2↑[(≢⍴w)-1]w)÷+/×⍨w←3⌽↑[(≢⍴⍵)-1+÷2]⍵
        s←A2/(⊂⍳2)∘⌷⍤1⊢w ⋄ d←A2⍨/(⊂2+⍳2)∘⌷⍤1⊢w ⋄ p1←0@(⊃0(○1)∨.=(⊂p)⍨)s-d
        p2←((⎕CT>|p)/⍥,2×s)@(⎕CT>∘|p⍨)((⎕CT>|p-○1)/⍥,2×d)@(⎕CT>∘|(○1)-⍨p⍨)s+d
        (○2)|⊂[(1-⍨≢⍴⍵)↓⍳(≢⍴p)]↑[⍳1]p1 p p2
    }
:EndNamespace

In [3]:
:Namespace EBSD                                                      ⍝ Read and write ang files
    Read←{'.ctf'≡⍥⎕C ¯4↑⍵: Ctf ⍵ ⋄ Ang ⍵}
    Ang←{                                                            ⍝ read ang file
        (h l)←((2-'#'=⊃¨)⊆⊢)⊃⎕NGET⍵1                                 ⍝   get (hash) comments and data
        c←⎕CSV⍠2⊢('(^\s+)|(\s+$)' '\s+'⎕R'' ','⊢l)'N'2 0             ⍝   format data and read as csv
        e←⎕NS⍬ ⋄ e.h←h ⋄ e.ea←↓⍉↑c[⍳3]                               ⍝   return namespace
        e.(x y ci iq p)←c[4 3 6 5 9] ⋄ e.ext←'ang'
        e.C←{⍉↑(↓⍉↑ea),y x iq ci 0 0 0} ⋄ e
    }
    Ctf←{                                                            ⍝ read ctf file
        R←{⊃⎕CSV(⊃↓/⍵.(Offsets,⊂Match))'S' 2}                        ⍝   read numeric value
        S←{⊃('^',⍺,'Step\s+(.*)')⎕S R ⍵}                             ⍝   step size from line
        l←(⎕UCS 10)(≠⊆⊢)80 ¯1⎕MAP ⍵ 'R' ⋄ x←y←0                      ⍝   read lines
        b←{'XStep'≡5↑⍵:0⊣x⊢←'X'S ⍵ ⋄ 'YStep'≡5↑⍵:0⊣y⊢←'Y'S ⍵         ⍝   read size
            ∧/⍵∊⎕D,' .-',⎕UCS 9}¨l~¨⎕UCS 13                          ⍝   lines with only digits
        c←⎕CSV⍠2⊢('(^\s+)|(\s+$)' '\s+'⎕R'' ','⊢b/l~¨⎕UCS 13)'N' 2 0 ⍝   read as csv
        e←⎕NS⍬ ⋄ e.h←(l/⍨~b) ⋄ e.ext←'ctf' ⋄ e.ea←180÷⍨○↓⍉↑c[5+⍳3]   ⍝   return namespace
        _←⎕SIGNAL(x≠y)/11  ⋄ e.(x y)←x y×c[1 2] ⋄ e.(ci iq)←c[8 9]
        e.C←{⍉↑÷∘(1⊃x)@1 2⊢0 x y 0 0,((180÷○1)×↓⍉↑ea),ci iq} ⋄ e
    }
    Crop←{                                                           ⍝ crop defined region
        (a b x y)←⍺ ⋄ (x y)←1e99@(=∘0)x y                            ⍝   default limits are 0 0 1e99 1e99
        e⊣(e←⎕NS ⍵).(ea x y iq ci)/⍨←⊂((a∘≤∧x∘>)⍵.x)∧(b∘≤∧y∘>)⍵.y    ⍝   select points between limits
    }
    Write←{                                                          ⍝ write comments and data to file
        f←⍵,'.',⍺.ext                                                ⍝   file name
        f⊣f 1⎕NPUT⍨⊂⍺.h,(⍺.C⍬)⎕CSV⍠2⍠'Separator' ' '⊢'' 'N'          ⍝   write header and data
    }
    Orientations←{                                                   ⍝ get orientations
        ⍵.d←⍵.x⌊⍥(⌈/⍤|2-/⊢)⍵.y
        nx←⌊0.5+1+((⌈/-⌊/)⍵.x)÷⍵.d ⋄ ny←⌊0.5+(≢⍵.x)÷nx               ⍝   find size
        ⍵.g←∪ea←(a←○⍺÷180)(⌊0.5+÷⍨)⍣(⍺>0)⊢(○2)|⍵.ea                  ⍝   unique euler angles
        (a×⍣(⍺>0)⊢⍵.g)(nx ny⍴⍵.g⍳ea)                                 ⍝   return angles and indices
    }
    Diameter←{2×(⍵÷○1)*÷2}∘{(⍳2)×.((≢,⍵){⍺⍺÷1⌈+/,⍵≠1⌽[⍺]⍵})⊂⍵}       ⍝ grain diameter (in cells)
:EndNamespace

In [4]:
]dinput
_SIM_←{
    o q ddis lagb hagb G0 M0 Qg A n flip f3 f4 fg dtmin kmodel dx←⍺
    CB←⍺⍺ ⋄ LF←⍵⍵
    N1←{(1⌽⍵)(1⊖1⌽⍵)(1⊖⍵)(1⊖¯1⌽⍵)}                              ⍝ e se s sw (from o)
    N2←{(¯1⊖¯1⌽⍵)(¯1⊖⍵)(¯1⊖1⌽⍵),(N1 ⍵),⊂¯1⌽⍵}                   ⍝ nw n ne e se s sw w (from o)
    N21←{e se s sw←⍵ ⋄ (¯2⊖¯2⌽se)(¯2⊖s)(¯2⊖2⌽sw),⍵,⊂¯2⌽e}       ⍝ nw n ne e se s sw w (from n1=e se s sw)
    D21←{e se s sw←⍵ ⋄ (¯1⊖¯1⌽se)(¯1⊖s)(¯1⊖1⌽sw),⍵,⊂¯1⌽e}       ⍝ nw n ne e se s sw w (from d1=de dse ds dsw)
    u←↓⍉↑|(⊢÷(÷2)*⍨+.×⍨¨)(1 ¯1)(0 ¯1)(¯1 ¯1)(¯1 0)(¯1 1)(0 1)(1 1)(1 0)      ⍝ unit vectors nw n ne e se s sw w
    mis←ddis Euler.M q ⋄ DL←mis.DL ⋄ D←(h←DL 180÷⍨○hagb)⌊mis.M               ⍝ disorientations
    TE←{⊃(1-⍺×1-fg)+.×⌷∘g¨¨⍵}                                                ⍝ transformation energy
    M←M0 Qg A n∘(h{m q a n←⍺ ⋄ (m×*-q÷8.314×⍵)×1-*-a×n*⍨⍺⍺÷⍨⍳1+⍺⍺})          ⍝ boundary mobility
    g←G0∘{(⍺×⊢×1-⍟)@(0∘<)¨1⌊⍵÷⍨⍳1+⍵}h                                        ⍝ boundary energy
    k←kmodel÷8××⍨dx ⋄ l←DL 180÷⍨○lagb
    STEP←{
        dt m←⍺ ⋄ o a t T←⍵ ⋄ dt-←t ⋄ dt≤0: o a t T ⋄ TL←{⍵+LF o ⍵ T t}TE
        b2 nn←{(∘.=⍨⍵)(3⌊1-⍨≢∘∪¨⊃,¨/⍵)}n2←N21⊢n1←N1 o
        f2 g2 m2←{(⍵∘.(∊¨)N2⊃,¨/⍵)(⊃+/⌷∘g¨¨⍵)(⌷∘m¨¨⍵)}D21⊢d1←(⊂o)D¨n1
        de dse ds dsw←d1 ⍝ (4)
        dee dsee dssee dsse dss←o∘D¨(2⌽o)(1⊖2⌽o)(2⊖2⌽o)(2⊖1⌽o)(2⊖o) ⍝ (9)
        tnw←f2[0;]TL ¯1⊖¨¯1⌽¨0 de dee dsee dssee dsse dss ds
        tse←f2[4;]TL 1⊖¨1⌽¨0(¯1⌽de)(¯2⌽dee)(¯1⊖¯2⌽dsee)(¯2⊖¯2⌽dssee)(¯2⊖¯1⌽dsse)(¯2⊖dss)(¯1⊖ds)
        dssw←o D 2⊖¯1⌽o ⍝ - ssee (9)
        tn←f2[1;]TL ¯1⊖¨(¯1⌽de)0 de dse dsse dss dssw dsw
        ts←f2[5;]TL 1⊖¨de 0(¯1⌽de)(¯1⊖¯1⌽dse)(¯2⊖¯1⌽dsse)(¯2⊖dss)(¯2⊖1⌽dssw)(¯1⊖1⌽dsw)
        dsww←o D 1⊖¯2⌽o ⍝ - sse (9)
        tw←f2[7;]TL ¯1⌽¨(¯1⊖ds)(¯1⊖1⌽dsw)(¯1⊖2⌽dsww)dee dsee dse ds 0
        te←f2[3;]TL 1⌽¨ds dsw dsww(¯2⌽dee)(¯1⊖¯2⌽dsee)(¯1⊖¯1⌽dse)(¯1⊖ds)0
        dssww←o D 2⊖¯2⌽o ⍝ - sw se see (7)
        tne←f2[2;]TL ¯1⊖¨1⌽¨(¯2⌽dee)(¯1⌽de)0 ds dss dssw dssww dsww
        tsw←f2[6;]TL 1⊖¨¯1⌽¨dee de 0(¯1⊖ds)(¯2⊖dss)(¯2⊖1⌽dssw)(¯2⊖2⌽dssww)(¯1⊖2⌽dsww)
        k2←(÷2)*⍨⊃+.×⍨+/¨u×⍤1¨⊂b2
        r←k×(⊂0 1 f3 f4[nn])×k2×m2×0⌈(⊂g2)-tnw tn tne te tse ts tsw tw   ⍝ transformation rates
        dt←{dt⌊dtmin⌈⌊/⍵÷⍨⍥((⍵>0)/∊)1-a}∊r                               ⍝ time increment
        a t+←(r×dt)dt                                                    ⍝ accumulators
        p←(⊃⍒)⍤1↑[-÷2]a                                                  ⍝ parents
        f←(⊢≥flip*⍨?⍤≠⍨)p⊃⍤0 1↑[-÷2]a                                    ⍝ cells to flip
        o←(⍴o)⍴(f/⍥,p⊃⍤0 1↑[-÷2]n2)@((,f)⍨),o                            ⍝ update orientations
        a←a×(⊂×N21⍤N1)~f                                                 ⍝ reset accumulators
        ⍺⍺ o a t T
    }
    S←{t T←⍺+(2⊃⍵)0 ⋄ t(M T)CB STEP⍣{t≤2⊃⍵}(3↑⍵),T}
    ⊃S/(⌽⍵),⊂CB o 0 0(⊃⌽⊃⍵)
}

In [5]:
]dinput
SIM←{
    0::⎕SIGNAL ⎕EN
    CUT←(1 1↓¯1 ¯1↓⊢)⍣⍺.edge
    EDGE←{m n←(⍴o←⍵)-1 ⋄ o[0 m;]←o[2(m-2);] ⋄ o[;0 n]←o[;2(n-2)] ⋄ o}⍣⍺.edge
    N2←(,2-⍳3 3){a b←⍺ ⋄ a⊖b⌽⍵}¨⊂
    FIX←{v←⍵≥0 ⋄ ((~v)/⍥,⍵⌷⍨∘(⊃⍤⍒)⍤1⍥(↑[-÷2]N2)(?≠⍨⍵)×v×⊃(⊂⍵)+.=N2 ⍵)@(~v⍨)⊢⍵}⍣≡
    CIRCLE←{o←(×⍨⍵)>+.×⍨¨(⍳2⍴s)+0.5(⊣-⊣×⊢)s←2×2×⍵ ⋄ q←(1 1 1 Euler.QAD 63.8)(4⍴0) ⋄ o q}
    RANDOM←{w h n←⍵ ⋄ q←↓⍉↑Euler.RND n ⋄ o←?h w⍴n ⋄ o q}
    INPUT←{                                                                      ⍝ microstructure
        2|⎕DR ⍵:{1=≢⍵:CIRCLE ⍵ ⋄ 3=≢⍵: RANDOM ⍵}⍵                                ⍝   generate
        ⍝ TODO: txt
        _←0 PRT ⍵,' ',⍕≢(e⊢←EBSD.Read ⍵).x ⋄ e⊢←⍺.crop EBSD.Crop⍣(0∨.≠⍺.crop)⊢e  ⍝   read and crop
        ea o←⍺.dang EBSD.Orientations e                                          ⍝   euler angles and micro
        (FIX ¯1@(⍺.(ci iq)∘{(⍴o)⍴⊃⍺∨.>e.(ci iq)})o)(↓⍉↑Euler.QE↓⍉↑ea)            ⍝   fixed micro and quaternions
    }
    CB←⍺∘{o a t T←⍵                                                              ⍝ callback
        td⍪←{⍵⊣⎕←12 ¯6⍕⍵}⍣(⍺.verbose>1)⊢t T,⍺.dx×EBSD.Diameter c←CUT o           ⍝   time and diameter
        _←{⍵.Callback c(CUT¨a)t}⍣(0≠⊃⍺.⎕NC'Callback')⊢⍺                          ⍝   user callback
        o←EDGE o ⋄ t<to+⍺.tout:o a t T                                           ⍝   fix edge
        to⊢←t ⋄ o a t T⊣1 PRT(WRT o to),' ',⍕≢,c                                 ⍝   write output
    }
    LF←⍺∘{o tg T t←⍵                                                             ⍝ local field
        ∆tg←⍺.(wkt×1.380649e¯23÷×⍨dx)×T×1-2×?≠⍨tg ⋄ ∆tg ⋄ 0∊⍴e:∆tg               ⍝   thermal noise
        ∆tg←⍺{⍵+⍺.wci×(1-e.ci⍴⍨⍴o)×1-2×?≠⍨⍵}⍣(⍺.wci>0)⊢tg                        ⍝   ci noise
        ⍺{⍵+⍺.iq×g×(⍴o)⍴1-(⊢÷⌈/)e.iq}⍣(⍺.iq>0)⊢∆tg                               ⍝   iq push
    }
    WRT←⍺.out{~0∊⍴e:EWRT ⍵ ⋄ o t←⍵ ⋄ f⊣(⊂↓⍕o)⎕NPUT(f←⍺⍺,'_','.txt',⍨1↓0 4⍕t)1}   ⍝ write output
    EWRT←⍺.out{o t←⍵ ⋄ e.ea←e.g[,|o] ⋄ e EBSD.Write ⍺⍺,'_',1↓0 4⍕t}              ⍝ write ebsd file
    PRT←(⍺.verbose>1){⎕←(35↑''),⍣⍺⍺⊢'<>'[⍺],' ',⍵}⍣(⍺.verbose>0)                 ⍝ print in(0)/out(1)
    e←⍬ ⋄ td←0 3⍴⍬ ⋄ o q←⍺ INPUT ⍺.input ⋄ _←1 PRT(WRT o 0),' ',⍕≢,o             ⍝ read input
    ⍺.dx←{e.d}⍣(⍺.dx=0)⊢⍺.dx ⋄ o←EDGE CUT⍣¯1⊢o                                   ⍝ init
    w←⍺.tout{                                                                    ⍝ output steps
        t←+\⍺{(⍺|⍵),⍨⍣(0≠⍺|⍵)⊢⍺/⍨⌊⍵÷⍺}⊃⌽w←+\⊃¨⍵                                  ⍝   time steps
        ↓⍉↑(-2-/0,{⍵[⍋⍵]}∪t,w)((⊃∘⌽¨⍵)[0,¯1↓(0,w)⍸{⍵[⍋⍵]}∪t,w])                  ⍝   temp at each step
    },⊆⍵
    to←0 ⋄ o a t T←w CB _SIM_ LF⍨o q,⍺.(ddis lagb hagb G0 M0 Qg A n flip f3 f4 fg dtmin kmodel dx)
    to←t-⍺.tout ⋄ _←⍺.out{(Euler.EA↓⍉↑⍵)⎕CSV⍠'IfExists' 'Replace'⊢⍺,'_ea.csv'}⍣(0∊⍴e)⊢q
    (CUT⊃CB⍣(0≠⍺.tout|⊃⌽⊃⌽w)⊢o a t T)td
}

In [6]:
:Namespace cfg
    out←'gg'              ⍝ output: prefix of output filenames
    tout←2.5              ⍝ output: time at which to write output filenames
    input←'ebsd.ang'      ⍝ input: filename
    input←10              ⍝ input: r for raious
    input←20 20 10        ⍝ input: w h n for random
    crop←1e¯7×2/5 30      ⍝ ebsd: crop region
    ci←1e¯2               ⍝ ebsd: confidence index
    iq←1e2                ⍝ ebsd: image quality
    dang←1                ⍝ ebsd: angular resolution (degrees)
    edge←1                ⍝ microstructure: edge (1) or periodic (0)
    dx←0.1e¯6             ⍝ microstructure: cell size
    lagb←5                ⍝ resolution: low angle grain boudary (degrees)
    hagb←15               ⍝ resolution: high angle grain boudary (degrees)
    ddis←0.5              ⍝ resolution: disorientations angular resolution (degrees)
    dtmin←0.01            ⍝ resolution: minimum time step
    kmodel←1              ⍝ transformation: rate scaling factor
    flip←0.1              ⍝ transformation: randomness of flips
    f3 f4←1.2 1.5         ⍝ transformation: triple and quadruple points
    fg←0.5                ⍝ transformation: geometric support factor
    wkt←0.1               ⍝ transformation: thermal noise factor
    wci←0.1               ⍝ transformation: confidence index noise factor
    wiq←0.1               ⍝ transformation: image quality noise factor
    G0←0.5                ⍝ physics: read-shockley maximum energy
    M0←1.5e¯6             ⍝ physics: humphrey's reference mobility
    Qg←140e3              ⍝ physics: humphrey's activation energy
    A←5                   ⍝ physics: humphrey's factor
    n←4                   ⍝ physics: humphrey's exponent
    verbose←1             ⍝ options: print time and diameter
:EndNamespace

In [7]:
⎕RL←1
cfg.input←'dc06.ctf' ⋄ cfg.dx←0 ⋄ cfg.verbose←2 ⋄ cfg.tout←100
cfg.Callback←{0=tout|2⊃⍵:⎕←⊃⍵ ⋄ ⍵}
o td←cfg SIM(250 750)(250 800) ⋄ o

< dc06.ctf 2500
 > gg_0.0000.ctf 625
0.00000E0 7.50000E2 1.30499E¯7 
 0 1 2 3 4 5 4 6 0 7 8 9 9 10 10 9 11 12 9 13 14 15 16 17 18
 19 20 21 0 0 0 4 4 22 23 24 25 26 27 28 29 28 30 29 10 29 31 32 18 18
 0 3 3 33 34 20 35 28 36 37 28 38 39 9 40 41 42 43 28 44 45 46 47 17 17
 3 48 0 49 50 49 51 9 12 40 42 40 52 9 27 41 42 11 13 29 53 54 18 55 17
 56 49 57 58 0 9 36 11 27 9 30 9 23 59 52 60 53 13 13 52 9 14 61 18 18
 62 74 0 0 9 14 44 9 14 14 36 65 9 14 41 45 30 65 41 28 9 60 66 55 17
 67 68 69 70 38 9 52 52 14 45 14 14 52 9 52 14 9 28 13 13 41 9 71 16 18
 69 72 73 74 74 9 52 45 75 13 52 14 76 11 14 52 65 9 41 12 41 13 77 55 17
 78 79 80 81 82 9 14 9 45 83 45 52 13 45 52 12 9 14 52 52 28 84 55 85 55
 69 74 86 87 69 88 9 11 52 9 45 23 30 89 14 41 45 52 52 41 9 9 90 18 18
 86 91 72 92 69 93 94 52 65 52 30 52 65 30 65 65 65 89 30 13 9 105 96 97 85
 69 78 69 87 98 99 100 101 36 13 52 9 52 14 52 14 14 14 30 12 103 103 104 105 69
107 108 109 109 110 100 111 111 111 112 111 113 14 89 41 30 30 52 14 103 103 105 103 114 115
116 117 118 101 100 100 116 116 119 119 120 121 121 121 13 30 52 103 103 103 96 103 105 103 114
123 117 111 124 120 116 116 120 125 111 111 111 111 119 126 28 103 103 128 105 103 115 129 103 130
109 116 116 131 116 116 116 111 109 109 123 116 111 111 119 119 112 115 103 103 115 103 103 103 103
111 116 116 116 116 109 116 125 111 111 111 111 119 116 116 111 132 105 133 103 105 105 103 105 103
125 125 111 100 109 116 111 116 123 109 116 123 100 119 116 111 121 134 135 105 105 103 103 115 105
111 121 116 111 111 111 124 123 123 100 111 136 123 100 111 116 119 137 138 105 103 115 103 105 115
119 125 137 100 111 100 111 111 100 121 111 111 121 111 116 132 111 139 140 141 141 133 105 103 103
111 119 123 109 111 111 116 123 100 121 111 111 132 111 111 111 143 144 145 145 140 105 103 103 115
121 124 116 116 111 111 121 111 136 123 109 111 111 120 146 111 147 143 148 140 141 145 140 115 105
111 111 124 111 111 111 111 111 111 149 134 137 150 151 152 140 154 155 141 141 156 141 154 142 157
118 111 111 137 119 111 116 111 119 132 111 158 159 160 161 162 144 140 141 141 156 145 140 141 111
164 124 111 111 119 111 119 119 165 166 166 167 151 168 162 169 170 140 141 141 156 140 141 141 171
1.52039E1 7.50000E2 1.30622E¯7 
2.79095E1 7.50000E2 1.30741E¯7 
3.13150E1 7.50000E2 1.30741E¯7 
3.26372E1 7.50000E2 1.30860E¯7 
3.96903E1 7.50000E2 1.31108E¯7 
4.96072E1 7.50000E2 1.31597E¯7 
5.25725E1 7.50000E2 1.31964E¯7 
5.65037E1 7.50000E2 1.32212E¯7 
6.62427E1 7.50000E2 1.32587E¯7 
6.78862E1 7.50000E2 1.32955E¯7 
7.12313E1 7.50000E2 1.33330E¯7 
8.12426E1 7.50000E2 1.33962E¯7 
8.70477E1 7.50000E2 1.34469E¯7 
9.13187E1 7.50000E2 1.34599E¯7 
9.75071E1 7.50000E2 1.35115E¯7 
1.00000E2 7.50000E2 1.35635E¯7 
 0 1 2 3 4 5 4 6 0 23 8 9 9 10 10 9 11 12 9 13 14 31 31 17 18
 19 20 21 0 0 0 4 4 23 23 24 25 26 27 28 29 28 30 29 10 29 31 32 18 18
 0 3 3 33 34 20 35 4 36 37 28 38 39 9 40 41 42 43 28 44 45 45 47 17 17
 3 48 0 49 50 49 51 9 12 40 42 40 52 9 27 41 42 11 13 29 53 54 18 55 17
 56 49 57 58 0 9 36 11 27 9 30 9 23 59 52 60 53 13 13 52 9 14 61 18 18
 74 74 0 0 9 14 44 9 14 14 36 65 9 14 41 45 30 65 41 28 9 60 61 55 17
 67 68 69 0 38 9 52 52 14 45 14 14 52 9 52 14 9 28 13 13 41 9 71 16 18
 69 72 73 74 9 9 52 45 75 13 52 14 76 11 14 52 65 9 41 12 41 13 55 55 17
 78 79 80 81 82 9 14 9 45 83 45 52 13 45 52 12 9 14 52 52 28 84 55 85 55
 69 74 86 87 69 88 9 11 52 9 45 23 30 89 14 41 45 52 52 41 9 9 18 18 18
 86 91 72 92 69 69 94 52 65 52 30 52 65 30 65 65 65 89 30 13 9 9 96 105 85
 69 78 109 109 98 99 100 101 111 111 52 9 52 14 52 14 14 14 30 103 103 103 104 105 105
108 108 109 109 110 100 111 111 111 112 111 113 121 89 41 30 30 52 103 103 103 105 103 114 115
116 117 118 101 100 100 116 116 119 119 120 121 121 121 13 30 103 103 103 103 96 103 105 103 114
123 117 111 124 120 116 116 120 125 111 111 111 111 119 126 119 103 103 128 105 103 115 129 103 130
109 116 116 131 116 116 116 111 109 109 123 116 111 111 119 119 103 115 103 103 115 103 103 103 103
111 11